It is I, again....

# Attention RNN with Sentence Embeddings

Trying our already computed trio word embeddings with a easy rnn with attention layer...

Steps:
- Loading the data.
    - Load answers to the tasks.
    - Load sentence embeddings.
    - Make sure they are the right order (how??? need to check old code, TODO)
- Preprocessing
    - Building correct arrays.
    - Train/Test splits.
- Building the Network
    - First a really simple one to get the parameters right.
    - Then we build a more complex one just for fun.
- Train the Network.
- Test the Network.
- Cry
    - because of overwhelming success
    - because of overwhelming failure (more likely)

In [1]:
# mf imports
import tensorflow as tf
import numpy as np
import pandas as pd

### Loading the Data

In [90]:
url_answers_task_A = "https://raw.githubusercontent.com/wangcunxiang/SemEval2020-Task4-Commonsense-Validation-and-Explanation/master/Training%20%20Data/subtaskA_answers_all.csv"

answers_task_A = pd.read_csv(url_answers_task_A, index_col=0, header=None)
answers_task_A = pd.DataFrame.to_numpy(answers_task_A)
print(answers_task_A.shape)
# index of the INVALID answer

(10000, 1)


In [97]:
sent_0 = np.load('first column trio embdes.npy')
sent_1 = np.load('second column trio embdes.npy')
print(sent_0.shape)
"""lets just assume they are in the right order, i mean...why shouldn't they :D ? /s...."""

(2, 10000, 50, 768)


"lets just assume they are in the right order, i mean...why shouldn't they :D ? /s...."

In [98]:
sent_0 = sent_0[0]
sent_1 = sent_1[1]

In [5]:
sent_0.shape

(10000, 50, 768)

### Preprocessing

Ok, now it is possible to build a network that takes two sentences as input and does something with them, but I dont want to read into that right now, so we'll use the standard way of classifying and give each sentence a True or False value, corresponding if it's valid or invalid. Easy peasy.

array([[1],
       [2],
       [3],
       [4],
       [5],
       [6]])

In [9]:
ex_0 = sent_0[0]
ex_1 = sent_1[0]
ex_0.shape

(50, 768)

In [7]:
data = np.concatenate((sent_0, sent_1))

In [10]:
ex_t = data[10000] # this is start of column sent_1
ex_z = data[10001]
print(np.array_equal(ex_1[0], ex_t[0]), 'earlier this returned false because it is very, very precise and we are using' \
      ' floats. \nCan use np.allclose though:', np.allclose(ex_1[0], ex_t[0]),
      'if thats not it maybe:', np.allclose(ex_1[0], ex_z[0]))


labels = np.zeros((20000,))
for i, idx_invalid in enumerate(answers_task_A):
    if idx_invalid == 0:
        labels[i] = 0
        labels[i+10000] = 1
    else:
        labels[i] = 1
        labels[i+10000] = 0

labels = np.asarray(labels)
print('data shape:',data.shape, 'labels shape:',labels.shape)
    
    

True earlier this returned false because it is very, very precise and we are using floats. 
Can use np.allclose though: True if thats not it maybe: False
data shape: (20000, 50, 768) labels shape: (20000,)


I'm pretty sure though that data should have a shape of (20000, n_feats), therefore:

#### Train - Test Split

In [11]:
from sklearn.model_selection import train_test_split
# takes a while, jesus these arrays are big
X_train, X_test, y_train, y_test = train_test_split(data, labels, test_size=0.33, random_state=42)

print('X_train shape:', X_train.shape, 'X_test shape:', X_test.shape, 'y_train shape:', y_train.shape
     , 'y_test shape:', y_test.shape)

X_train shape: (13400, 50, 768) X_test shape: (6600, 50, 768) y_train shape: (13400,) y_test shape: (6600,)


## Building the Network

### First a simple dnn



In [17]:
"""quick check how long the longest sequence actually is..."""
highest = 0
for line in data:
    for idx, trio in enumerate(line):
        #print(trio)
        if trio[0] == 0.:
            if idx > highest:
                highest = idx

In [18]:
print(highest) # oh... then 50 is a pretty good padding lol

49


#### Using single word embeddings for vanilla- network

In [ ]:
single_word_embs = np.load('single_word_embs.npy')

In [ ]:
sent_0 = single_word_embs[0]
sent_1 = single_word_embs[1]
data = np.concatenate((sent_0, sent_1))


In [23]:
import tensorflow.keras as keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Layer, Input, Dense, Activation, Conv2D, \
                        Dropout, Flatten, MaxPooling2D, Bidirectional, LSTM, Bidirectional
from keras import initializers, regularizers, constraints, optimizers, layers
#from keras import backend as K
model = Sequential()



model.add(Bidirectional(LSTM(256, return_sequences=True, 
               dropout=0.1, recurrent_dropout=0.1)))
model.add(LSTM(256, return_sequences=False, 
               dropout=0.1, recurrent_dropout=0.1))


model.add(Dense(64, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.2))

# Output layer
#model.add(Dense(2, activation='softmax'))
model.add(Dense(1, activation='sigmoid'))

# Compile the model
model.compile(
    optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [24]:
model.fit(X_train, y_train,
          batch_size=64,
          epochs=10,
          verbose=1,
          validation_data=(X_test, y_test))
score = model.evaluate(X_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Train on 13400 samples, validate on 6600 samples
Epoch 1/10
13400/13400 [==============================] - 402s 30ms/sample - loss: 5.9427e-08 - accuracy: 0.5013 - val_loss: 5.9966e-08 - val_accuracy: 0.4970
Epoch 2/10
13400/13400 [==============================] - 400s 30ms/sample - loss: 5.9427e-08 - accuracy: 0.4993 - val_loss: 5.9966e-08 - val_accuracy: 0.4970
Epoch 3/10
 7168/13400 [===============>..............] - ETA: 3:48 - loss: 5.8799e-08 - accuracy: 0.5065

KeyboardInterrupt: 

In [25]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
bidirectional_2 (Bidirection multiple                  2099200   
_________________________________________________________________
lstm_5 (LSTM)                multiple                  787456    
_________________________________________________________________
dense_6 (Dense)              multiple                  16448     
_________________________________________________________________
dropout_4 (Dropout)          multiple                  0         
_________________________________________________________________
dense_7 (Dense)              multiple                  4160      
_________________________________________________________________
dropout_5 (Dropout)          multiple                  0         
_________________________________________________________________
dense_8 (Dense)              multiple                 

In [58]:
### Try siamese network with functional api
from keras.models import Model
import tensorflow.keras as keras
#import tensorflow_probability
#tensorflow_probability=='0.8.0rc0'
from tensorflow.keras.layers import Layer, Input, Dense, Activation, Conv2D, \
                        Dropout, Flatten, MaxPooling2D, Bidirectional, LSTM
from keras import initializers, regularizers, constraints, optimizers, layers
"""
inp_0 = Input(shape=sent_0[0].shape)
x_0 = Bidirectional(LSTM(64, return_sequences=True,dropout=0.1, recurrent_dropout=0.1))(inp_0)
x_0 = Bidirectional(LSTM(64, return_sequences=True,dropout=0.1, recurrent_dropout=0.1))(x_0)
x_0 = Dense(64, activation='relu')(x_0)
x_0 = Dropout(0.1)(x_0)

inp_1 = Input(shape=sent_1[0].shape)
x_1 = Bidirectional(LSTM(64, return_sequences=True,dropout=0.1, recurrent_dropout=0.1))(inp_1)
x_1 = Bidirectional(LSTM(64, return_sequences=True,dropout=0.1, recurrent_dropout=0.1))(x_1)
x_1 = Dense(64, activation='relu')(x_1)
x_1 = Dropout(0.1)(x_1)
"""
inp_0 = Input(shape=sent_0[0].shape)
inp_1 = Input(shape=sent_1[0].shape)

# Instantiate the layers
lstm_1 = LSTM(64, return_sequences=True, dropout=0.1, recurrent_dropout=0.1)
lstm_2 = LSTM(128, return_sequences=True, dropout=0.1, recurrent_dropout=0.1)
lstm_3 = LSTM(256, return_sequences=True, dropout=0.1, recurrent_dropout=0.1)
lstm_4 = LSTM(128, return_sequences=False, dropout=0.1, recurrent_dropout=0.1)
fc_1 = Dense(128, activation='relu')
dropout_1 = Dropout(0.1)
fc_2 = Dense(128, activation='relu')
dropout_2 = Dropout(0.1)
fc_3 = Dense(128, activation='relu')
dropout_3 = Dropout(0.1)

x_0 = lstm_1(inp_0)
#x_0 = lstm_2(x_0)
#x_0 = lstm_3(x_0)
x_0 = lstm_4(x_0)
x_0 = fc_1(x_0)
x_0 = dropout_1(x_0)
x_0 = fc_2(x_0)
x_0 = dropout_2(x_0)
x_0 = fc_3(x_0)
x_0 = dropout_3(x_0)

x_1 = lstm_1(inp_1)
#x_1 = lstm_2(x_1)
#x_1 = lstm_3(x_1)
x_1 = lstm_4(x_1)
x_1 = fc_1(x_1)
x_1 = dropout_1(x_1)
x_1 = fc_2(x_1)
x_1 = dropout_2(x_1)
x_1 = fc_3(x_1)
x_1 = dropout_3(x_1)


#x = keras.layers.concatenate([x_0, x_1]) # for first try with two sets of seperately defined layers
x = keras.layers.concatenate([x_0, x_1], axis=-1)
"""
x = Dense(64, activation='relu')(x)
x = Dropout(0.2)(x)
x = Dense(64, activation='relu')(x)
x = Dropout(0.2)(x)
"""
out = Dense(2, activation='softmax')(x)

model = tf.compat.v1.keras.Model(inputs=[inp_0,inp_1], outputs=out)
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [48]:
from keras.utils import to_categorical
labels = answers_task_A.reshape((10000))
print(labels.shape)
labels = to_categorical(labels)
#print(y_binary.shape)
from sklearn.model_selection import train_test_split
# need to think for a good split method for sent_0, snet_1
#X_train, X_test, y_train, y_test = train_test_split(data, labels, test_size=0.33, random_state=42)

(10000,)


In [59]:
model.summary()

Model: "model_9"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_25 (InputLayer)           [(None, 50, 768)]    0                                            
__________________________________________________________________________________________________
input_26 (InputLayer)           [(None, 50, 768)]    0                                            
__________________________________________________________________________________________________
lstm_28 (LSTM)                  (None, 50, 64)       213248      input_25[0][0]                   
                                                                 input_26[0][0]                   
__________________________________________________________________________________________________
lstm_31 (LSTM)                  (None, 128)          98816       lstm_28[0][0]              

In [60]:
model.fit([sent_0,sent_1], labels, batch_size=128, epochs=5, verbose=1 )

Train on 10000 samples
Epoch 1/5
10000/10000 [==============================] - 75s 8ms/sample - loss: 0.6934 - accuracy: 0.4918
Epoch 2/5
10000/10000 [==============================] - 64s 6ms/sample - loss: 0.6934 - accuracy: 0.4968
Epoch 3/5
10000/10000 [==============================] - 59s 6ms/sample - loss: 0.6934 - accuracy: 0.4956
Epoch 4/5
10000/10000 [==============================] - 58s 6ms/sample - loss: 0.6932 - accuracy: 0.5080
Epoch 5/5
10000/10000 [==============================] - 59s 6ms/sample - loss: 0.6933 - accuracy: 0.4953


In [101]:
"""mad genius approach"""

from keras.models import Model
import tensorflow.keras as keras
#import tensorflow_probability
#tensorflow_probability=='0.8.0rc0'
from tensorflow.keras.layers import Layer, Input, Dense, Activation, Conv2D, \
                        Dropout, Flatten, MaxPooling2D, Bidirectional, LSTM
from keras import initializers, regularizers, constraints, optimizers, layers
from sklearn.metrics.pairwise import cosine_similarity

inp_0 = Input(shape=sent_0[0].shape)
#inp_1 = Input(shape=sent_0[0].shape)
bilstm = Bidirectional(LSTM(768, return_sequences=True, return_state=True))
#lstm = LSTM(768, return_sequences=True)
# hidden state sequence, last hidden state forward, last hidden cell forward, last hidden state backward, l.h. cel backward
hs_seq, ls_fw, lc_fw, ls_bw, lc_bw = bilstm(inp_0)
#hs_seq = lstm(inp_0)
#hs_seq = bilstm(inp_0)


model = tf.compat.v1.keras.Model(inputs=inp_0, outputs=[hs_seq, ls_fw, lc_fw, ls_bw, lc_bw])
#model = tf.compat.v1.keras.Model(inputs=inp_0, outputs=[hs_seq,ls_fw,ls_bw])
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model.summary()




Model: "model_7"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_19 (InputLayer)        [(None, 50, 768)]         0         
_________________________________________________________________
bidirectional_20 (Bidirectio [(None, 50, 1536), (None, 9443328   
Total params: 9,443,328
Trainable params: 9,443,328
Non-trainable params: 0
_________________________________________________________________


In [76]:
for sent in sent_0:
    print(sent.shape)
    output = model.predict(sent.reshape((1,50,768)))
    break
print(output)

(50, 768)
[array([[[ 9.98825114e-03,  1.73072349e-02, -8.15293044e-02, ...,
          1.96081191e-01,  5.92003204e-02,  1.95593104e-01],
        [ 4.75729518e-02, -5.16957343e-02, -1.09136216e-01, ...,
          2.58895159e-01,  1.07234061e-01,  2.44306028e-01],
        [ 3.43362056e-02, -4.91204187e-02, -2.07990274e-01, ...,
          2.15114072e-01,  5.79758957e-02,  2.32335329e-01],
        ...,
        [ 1.59235478e-05, -1.61200514e-04, -2.70023651e-04, ...,
          0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
        [ 4.10183566e-06, -1.38541727e-04, -2.28758814e-04, ...,
          0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
        [-4.21483401e-06, -1.19011645e-04, -1.92978652e-04, ...,
          0.00000000e+00,  0.00000000e+00,  0.00000000e+00]]],
      dtype=float32), array([[-4.21483401e-06, -1.19011645e-04, -1.92978652e-04,
         1.34796108e-04,  5.21785551e-05, -7.75208027e-05,
         2.78542691e-04, -6.55235053e-05, -1.71773790e-05,
        -2.39171597

In [77]:
from sklearn.metrics.pairwise import cosine_similarity

hs_seq = output[0]
ls_fw = output[1]
lc_fw = output[2]
ls_bw = output[3]
lc_bw = output[4]
print(hs_seq.shape, ls_fw.shape, lc_fw.shape)
print(cosine_similarity(ls_fw, ls_bw))
#print(cosine_similarity(hs_seq[0], ls_fw))
#print(cosine_similarity(hs_seq[0], ls_bw))

(1, 50, 1536) (1, 768) (1, 768)
[[-0.02247526]]


In [110]:
output = model.predict(sent_0[4].reshape((1,50,768)))

ls_fw_0 = output[1]
ls_bw_0 = output[3]
print(cosine_similarity(ls_fw_0, ls_bw_0))
output = model.predict(sent_1[4].reshape((1,50,768)))
ls_fw_1 = output[1]
ls_bw_1 = output[3]
print(cosine_similarity(ls_fw_1, ls_bw_1))

#print(cosine_similarity(hs_seq[0], ls_fw))
#print(cosine_similarity(hs_seq[0], ls_bw))

#print(cosine_similarity(ls_fw_0, ls_fw_1))
#print(cosine_similarity(ls_bw_0, ls_bw_1))

#print(cosine_similarity(ls_fw_0, ls_bw_1))
#print(cosine_similarity(ls_bw_0, ls_fw_1))

[[-0.04253481]]
[[-0.04104108]]


In [112]:

tuple_of_cos = []

for s0, s1 in zip(sent_0,sent_1):
    out0 = model.predict(s0.reshape((1,50,768)))
    out1 = model.predict(s1.reshape((1,50,768)))
    cos0 = cosine_similarity(out0[1],out0[3])
    cos1 = cosine_similarity(out1[1],out1[3])
    tuple_of_cos.append((cos0,cos1))


KeyboardInterrupt: 

In [113]:
len(tuple_of_cos)

2818

In [114]:
valid_bigger = 0
invalid_bigger = 0

for cosines, inv_idx in zip(tuple_of_cos, answers_task_A):
    if inv_idx == 0:
        if cosines[0] < cosines[1]:
            valid_bigger += 1
        elif cosines[0] > cosines[1]:
            invalid_bigger += 1
    elif inv_idx == 1:
        if cosines[0] < cosines[1]:
            invalid_bigger += 1
        elif cosines[0] > cosines[1]:
            valid_bigger += 1
            
print(valid_bigger, invalid_bigger)

1351 1467


In [ ]:
#TODO: make new dataset with valid, valid mixed into !!!
# also train normal lstm with just sentence vectors -- bert implementation word vectors or just with an embedding layer?

#TODO: find max trio seq length, make better embeddings
#TODO: just use colab duuuudee
# Is your implementation of lstm really correct? input dimension correct? what should the input array be made of?